In [0]:


import datetime
import pandas as pd

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


def get_date_granularity_mapping_list(begin_date, end_date):
    date_granularity_mapping_list = {
        "monthly": get_date_list(begin_date, end_date, "M"),
        "weekly": get_date_list(begin_date, end_date, "W-SAT")
    }
    return date_granularity_mapping_list

begin_date = datetime.datetime.strptime("2021-06-19", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2021-07-15", '%Y-%m-%d')


DATE_DICT = get_date_granularity_mapping_list(begin_date, end_date)
range_type_mapping = {"daily":"DAY","monthly":"MONTH","weekly":"WEEK"}


for granularity in DATE_DICT:
    for date in DATE_DICT[granularity]:
        
        try:
            s3path = "s3://aardvark-prod-pdx-mdm-to-int/to_tech/demographics/version=ios_diamond_1.0.0/range_type={range_type}/date={date}/".format(range_type=range_type_mapping[granularity], date=date)
            spark.read.parquet(s3path).createOrReplaceTempView("test_demographics")
            df = spark.sql("select  '{granularity}' as granularity, '{date}' as date, count(distinct(app_id)) as count_app_id, count(distinct(country)) as count_country, count(distinct(device_type)) as count_device_type,count(distinct(age)) as count_age, count(distinct(gender)) as count_gender, sum(AU) as sum from test_demographics".format(date=date, granularity=granularity))
            df.write.format("parquet").save("s3://b2c-prod-data-pipeline-qa/aa.tom.s6usage/2021-07-05/demographics/", mode="append") #append
            print("PASS on {} {}".format(granularity, date))
        except Exception as e:
            print("ERROR on {} {}".format(granularity, date))


In [0]:
%%sh
